# Machine Learning on Text Data

In this tutorial, we are going to explore, how ML techniques can be utilized to build a simple text classifier, that can perform sentiment analysis. The idea is very straightforward. We first build one hot encoded representation of our dataset, and then train and evaluate ML models on the one hot vectors of the samples.

IMDB Movie Review Dataset Source: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
# Connect the Google drive, where our data is present
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Read the data

import pandas as pd

dataDF = pd.read_csv('/content/drive/My Drive/GUIST_Webinar_Files/Jupyter_Notebook_Codes/Data/IMDB_Dataset.csv')

print(dataDF.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
print(dataDF.iloc[0]['review'])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [5]:
print(len(dataDF))

50000


In [6]:
allData = list()
allLabels = list()

for i in range(5000):
  allData.append(dataDF.iloc[i]['review'])
  if dataDF.iloc[i]['sentiment'] == 'positive':
    allLabels.append(1)
  else:
    allLabels.append(0)

print(len(allData))
print(len(allLabels))
print(allData[0])

5000
5000
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due 

Now we have our text data and their labels. However, in case of text data, there is usually a fair amount of preprocessing is needed to get the data ready for ML.

So, first we are going to perform some simple cleaning, and then we will proceed for ML classification.

In [7]:
# Let's look at some of the reviews and see what kinds of cleaning could be needed...
for i in range(10):
  print(allData[i])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

Looks like, we may need to remove the break tags. Furthermore removing very frequently occurring uninformative stopwords like the, a, of, etc. could further make things easier for our classifiers.


In [8]:
# Remove the <br /> tags using string replace function
for i in range(len(allData)):
  allData[i] = allData[i].replace('<br />',' ')

# Check if the removal worked or not
for i in range(10):
  print(allData[i])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.  The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.  It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.  I would say the main appeal of the show is due to the fact that it goes where other sho

In [9]:
# To make things simple, we will replace all the punctuations also

import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space

for i in range(len(allData)):
  allData[i] = allData[i].translate(translator)

# Check if the cleaning worked

for i in range(10):
  print(allData[i])

One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me   The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word   It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away   I would say the main appeal of the show is due to the fact that it goes where other sho

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english')) 

for i in range(len(allData)):
  # Break the review into a list of words
  word_tokens = word_tokenize(allData[i]) 
  
  # Remove the stopwords from the list
  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  # Rejoin the list of words into a review string
  allData[i] = ' '.join(filtered_sentence)

# Check if the stopword removal worked or not
for i in range(10):
  print(allData[i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
One reviewers mentioned watching 1 Oz episode hooked They right exactly happened The first thing struck Oz brutality unflinching scenes violence set right word GO Trust show faint hearted timid This show pulls punches regards drugs sex violence Its hardcore classic use word It called OZ nickname given Oswald Maximum Security State Penitentary It focuses mainly Emerald City experimental section prison cells glass fronts face inwards privacy high agenda Em City home many Aryans Muslims gangstas Latinos Christians Italians Irish scuffles death stares dodgy dealings shady agreements never far away I would say main appeal show due fact goes shows dare Forget pretty pictures painted mainstream audiences forget charm forget romance OZ mess around The first episode I 

As it is clear that removing the punctuations did introduce some linguistic problems, however we will move forward with the vocabulary creation now, which will help us with one hot vectorizing the strings.

In [11]:
# Join all reviews
giantString = ' '.join(allData)

# Get all the words from the reviews combined, as a list
allWords = giantString.split()

# Make a unique words dictionary
allWordsDict = {}

for i in range(len(allWords)):
  allWordsDict[allWords[i]] = 1

# Create a unique words vocabulary
vocabList = list()

for key, value in allWordsDict.items():
  vocabList.append(key)

print('Number of unique words in the vocabulary: ', len(vocabList))
for i in range(100):
  print(vocabList[i])

Number of unique words in the vocabulary:  46738
One
reviewers
mentioned
watching
1
Oz
episode
hooked
They
right
exactly
happened
The
first
thing
struck
brutality
unflinching
scenes
violence
set
word
GO
Trust
show
faint
hearted
timid
This
pulls
punches
regards
drugs
sex
Its
hardcore
classic
use
It
called
OZ
nickname
given
Oswald
Maximum
Security
State
Penitentary
focuses
mainly
Emerald
City
experimental
section
prison
cells
glass
fronts
face
inwards
privacy
high
agenda
Em
home
many
Aryans
Muslims
gangstas
Latinos
Christians
Italians
Irish
scuffles
death
stares
dodgy
dealings
shady
agreements
never
far
away
I
would
say
main
appeal
due
fact
goes
shows
dare
Forget
pretty
pictures
painted
mainstream
audiences
forget


Next we convert all the data that we have to one hot vectors, with respect to our vocabulary.

In [12]:
allNumData = list()

for i in range(len(allData)):
  words = allData[i].split()
  tempList = list()
  for j in range(len(vocabList)):
    if vocabList[j] in words:
      tempList.append(1)
    else:
      tempList.append(0)
  allNumData.append(tempList)

print(len(allNumData))
print(len(allLabels))
print(allNumData[0])

5000
5000
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Now, we have numerical data and labels, so we can proceed towards ML classification.

In [14]:
from sklearn.model_selection import train_test_split

# 80-20 Split, in a stratified manner
xTrain, xTest, yTrain, yTest = train_test_split(allNumData, allLabels, test_size=0.2, shuffle = True, stratify = allLabels)

In [15]:
# Import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Create a list of models
models = []
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVC', SVC(max_iter = 100000)))
models.append(('LR', LogisticRegression(max_iter = 100000)))
models.append(('DT', DecisionTreeClassifier()))
models.append(('GNB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))
models.append(('GB', GradientBoostingClassifier()))

# Classify and evaluate
names = []
scores = []
for name, model in models:
    model.fit(xTrain, yTrain)
    y_pred = model.predict(xTest)
    scores.append(accuracy_score(yTest, y_pred))
    names.append(name)

# Print the results in a pretty manner, using Pandas
tr_split = pd.DataFrame({'Name': names, 'Score': scores})
print(tr_split)

  Name  Score
0  KNN  0.562
1  SVC  0.857
2   LR  0.869
3   DT  0.698
4  GNB  0.626
5   RF  0.841
6   GB  0.806
